In [19]:
import numpy as np
import cv2
import open3d as o3d
import time

# 读取深度图像
depth_img = cv2.imread("../img/1_depth.png", cv2.IMREAD_GRAYSCALE)

In [68]:
h, w = depth_img.shape
# 相机内参矩阵
k = np.array([[535.4, 0.0, 320.0],
              [0.0, 539.2, 240.0],
              [0.0, 0.0, 1.0]])

# 每次都一样，避免重复计算
i, j = np.indices((h, w))
static_array = np.stack((i, j, np.ones((h, w))), axis=2).reshape(-1, 3)
# 获取相机内参的逆矩阵
k_inv = np.linalg.inv(k)
point_array = np.dot(k_inv, np.vstack([np.indices((w, h)).reshape(2, -1), np.ones((1,w * h))]))

def depth_to_point_cloud_v4(depth_img):
    # 使用矩阵乘法计算每个像素点的3D坐标
    point = point_array * depth_img.reshape(-1).T
    return point

In [32]:
a = np.indices((w, h)).reshape(2, -1)
b = np.ones((1,w * h))
np.vstack((a, b))

# stack是形成新维度，hstack不会形成新维度
# np.stack((a, b), axis=1)

array([[  0.,   0.,   0., ..., 479., 479., 479.],
       [  0.,   1.,   2., ..., 637., 638., 639.],
       [  1.,   1.,   1., ...,   1.,   1.,   1.]])

In [69]:
# 十次求平均
elapsed_time_list = []
for i in range(10):# 
    start_time = time.time()  # 记录开始时间

    # 将深度图转换为点云
    pc = depth_to_point_cloud_v4(depth_img)

    end_time = time.time()  # 记录结束时间

    elapsed_time = end_time - start_time  # 计算耗时

    elapsed_time_list.append(elapsed_time)

average_time = sum(elapsed_time_list) / len(elapsed_time_list)

print(f"Time elapsed average 10 times:{average_time:.4f} seconds.")

Time elapsed average 10 times:0.0023 seconds.


In [71]:
import cProfile
cProfile.run('depth_to_point_cloud_v4(depth_img)')

         5 function calls in 0.003 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 26705129.py:14(depth_to_point_cloud_v4)
        1    0.000    0.000    0.003    0.003 <string>:1(<module>)
        1    0.000    0.000    0.003    0.003 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}




In [67]:
# 可视化点云

# 点云数据存储在名为point_cloud的numpy数组中
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(pc)

# 创建一个可视化窗口
vis = o3d.visualization.Visualizer()
vis.create_window()

# 将点云添加到可视化窗口中
vis.add_geometry(cloud)

# 设置相机位置，没调所以可有可无
vis.get_view_control().set_front([0, 0, -1])
vis.get_view_control().set_lookat([0, 0, 0])
vis.get_view_control().set_up([0, -1, 0])

# 开始显示点云
vis.run()

# 关闭显示窗口
vis.destroy_window()

0.0027s! 简化了代码